Rayleigh-Taylor instability
======

This notebook models the Rayleigh-Taylor instability outlined in van Keken *et al.* (1997). 

The system of equations is given by

$$
    \nabla \cdot \left( \eta \nabla \dot\varepsilon \right) - \nabla p = -\rho g \mathbf{\hat z}
$$

$$
    \nabla \cdot \mathbf{v} = 0
$$

**New concepts:** particle swarms, Stokes system, Mapping underworld.functions

**References**

1. van Keken, P.E., S.D. King, H. Schmeling, U.R. Christensen, D.Neumeister and M.-P. Doin. A comparison of methods for the modeling of thermochemical convection. Journal of Geophysical Research, 102, 22,477-22,495, 1997.  
http://onlinelibrary.wiley.com/doi/10.1029/97JB01353/abstract

<img src='../images/raytay.png' width=600>


In [ ]:
import underworld as uw
from underworld import function as fn
import glucifer
import math
import numpy as np

In [ ]:
outputPath = 'output'

# create outputPath
import glob, json, os
# make a unique path
if os.path.exists(outputPath):
    outputPath += '_'+str(len(glob.glob(outputPath+str('*')))-1)

# build output dir string
if not outputPath.endswith('/'):
    outputPath += '/'
    
# make the output path
if not os.path.exists(outputPath):
    os.makedirs(outputPath)

Setup parameters
-----

In [ ]:
res = 64
boxLength = 0.9142
boxHeight = 1.0

Create mesh and finite element variables
------

In [ ]:
mesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q1/dQ0"), 
                                 elementRes  = (res, res), 
                                 minCoord    = (0., 0.), 
                                 maxCoord    = (boxLength, boxHeight))

velocityField = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=2 )
pressureField = uw.mesh.MeshVariable( mesh=mesh.subMesh, nodeDofCount=1 )

# functions to caluculate strain rate second invariant
strainRate = fn.tensor.symmetric( velocityField.fn_gradient )
strainRateInv = fn.tensor.second_invariant( strainRate )

# initialise 
velocityField.data[:] = [0.,0.]
pressureField.data[:] = 0.

Create a particle swarm
------

In [ ]:
# Create a swarm.
swarm = uw.swarm.Swarm( mesh=mesh )

# Create a data variable. It will be used to store the material index of each particle.
materialIndex = swarm.add_variable( dataType="int", count=1 )

# Create a layout object, populate the swarm with particles.
swarmLayout = uw.swarm.layouts.GlobalSpaceFillerLayout( swarm=swarm, particlesPerCell=20 )
swarm.populate_using_layout( layout=swarmLayout )

Initialise each particle's material index
-----

In [ ]:
# define these for convience. 
denseIndex = 0
lightIndex = 1

# material perturbation from van Keken et al. 1997
wavelength = 2.0*boxLength
amplitude  = 0.02
offset     = 0.2
k = 2. * math.pi / wavelength

# Create function to return a coordinate
coord = fn.coord()

# Define the material perturbation, a function of the x coordinate (accessed by `coord[0]`).
perturbationFn = offset + amplitude*fn.math.cos( k*coord[0] )

# Setup the conditions list. 
# If z is less than the perturbation, set to lightIndex.
conditions = [ ( perturbationFn > coord[1] , lightIndex ),
               (                      True , denseIndex ) ]

# The swarm is passed as an argument to the evaluation, providing evaluation on each particle.
# Results are written to the materialIndex swarm variable.
materialIndex.data[:] = fn.branching.conditional( conditions ).evaluate(swarm)

**Plot the strain rate invariant**

In [ ]:
fig = glucifer.Figure()
fig.append(glucifer.objects.Surface(mesh, strainRateInv))
fig.show()

**Plot the particles by material**

In [ ]:
fig1 = glucifer.Figure()
fig1.append( glucifer.objects.Points(swarm, materialIndex, pointSize=2, colourBar=False) )
fig1.append( glucifer.objects.VectorArrows( mesh, velocityField, scaling=1e1))
fig1.show()

Map properties to material index
-----
The Map function allows us to create 'per material' type behaviour. Again we use the branching function to set up a (condition, action) command. 

In [ ]:
# Set a density of '0.' for light material, '1.' for dense material.
densityMap   = { lightIndex:0., denseIndex:1. }
densityFn    = fn.branching.map( fn_key = materialIndex, mapping = densityMap )

# Set a viscosity value of '1.' for both materials.
viscosityMap = { lightIndex:1., denseIndex:1. }
fn_viscosity  = fn.branching.map( fn_key = materialIndex, mapping = viscosityMap )

# Define a vertical unit vector using a python tuple.
z_hat = ( 0.0, 1.0 )

# Create buoyancy force vector
buoyancyFn = -densityFn*z_hat

Boundary conditions
-----

Create free-slip condition on the vertical boundaries, and a no-slip condition on the horizontal boundaries.

In [ ]:
# Construct node sets using the mesh specialSets
iWalls = mesh.specialSets["MinI_VertexSet"] + mesh.specialSets["MaxI_VertexSet"]
jWalls = mesh.specialSets["MinJ_VertexSet"] + mesh.specialSets["MaxJ_VertexSet"]
allWalls = iWalls + jWalls

# Prescribe degrees of freedom on each node to be considered Dirichlet conditions.
# In the x direction on allWalls flag as Dirichlet
# In the y direction on jWalls (horizontal) flag as Dirichlet
stokesBC = uw.conditions.DirichletCondition( variable      = velocityField, 
                                             indexSetsPerDof = (allWalls, jWalls) )

Create systems
-----

In [ ]:
stokes = uw.systems.Stokes( velocityField = velocityField, 
                            pressureField = pressureField,
                            voronoi_swarm = swarm, 
                            conditions    = stokesBC,
                            fn_viscosity  = fn_viscosity, 
                            fn_bodyforce  = buoyancyFn )

solver = uw.systems.Solver( stokes )

# Optional solver settings
if(uw.nProcs()==1):
    solver.set_inner_method("lu")
    solver.set_penalty(1.0e6) 
solver.options.scr.ksp_rtol = 1.0e-3

# Create a system to advect the swarm
advector = uw.systems.SwarmAdvector( swarm=swarm, velocityField=velocityField, order=2 )

Time stepping
-----

In [ ]:
# Initialise time and timestep.
time = 0.
step = 0

# We set timeEnd=3 so that the simulation completes quickly, 
# but generally you will want to set timeEnd~300 to capture 
# the peak V_rms. 
timeEnd      = 150.  
outputEvery  = 3
timeVal     = []
vrmsVal     = []

# functions for calculating RMS velocity
vdotv = fn.math.dot(velocityField,velocityField)
v2sum_integral  = uw.utils.Integral( mesh=mesh, fn=vdotv )
volume_integral = uw.utils.Integral( mesh=mesh, fn=1. )

outfile = open(outputPath+'time.data', 'w+')
string = "steps, time, vrms"
print(string)
outfile.write( string+"\n")

# Save mesh and retain file handle for future xdmf creation
meshFileHandle = mesh.save(outputPath+"Mesh.h5")

In [ ]:
# define an update function
def update():
    # Retrieve the maximum possible timestep for the advection system.
    dt = advector.get_max_dt()
    # Advect using this timestep size.
    advector.integrate(dt)
    return time+dt, step+1

In [ ]:
while time < timeEnd:
    # Get instantaneous Stokes solution
    solver.solve()
    
    # Calculate the RMS velocity.
    vrms = math.sqrt( v2sum_integral.evaluate()[0] / volume_integral.evaluate()[0] )
    # Record values into arrays
    if(uw.rank()==0):
        string = 'step = {0:6d}; time = {1:.3e}; v_rms = {2:.3e}'.format(step,time,vrms)      
        print(string)
        outfile.write(string+"\n")
    
    # Output to disk
    if step%outputEvery == 0:
        velFilename = outputPath+"/velocityField."+str(step)
        vFH = velocityField.save(velFilename+".h5")
        velocityField.xdmf( velFilename, vFH, "velocity", meshFileHandle, "Mesh", time )
        
        outputFilename = outputPath+"sr_inv"+str(step).zfill(4)
        fig.save_image(outputFilename)
        
        outputFilename = outputPath+"image"+str(step).zfill(4)
        fig1.save_image(outputFilename)

    # We are finished with current timestep, update.
    time, step = update()
    
if(uw.rank()==0):
    outfile.close()
    print 'step = {0:6d}; time = {1:.3e}; v_rms = {2:.3e}'.format(step,time,vrms)

In [ ]:
fig1.show()

In [ ]:
fig.show()

**Exercise**

1) Experimental with a different material distribution

2) Experimental with the gravity field magnitude, how does the time scale change and why?
   (Consider limiting the timestep)